In [1]:
import os
import datetime

from utils.visualization import plot_predictions

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sqlalchemy import create_engine, sql

In [2]:
# matplotlib settings
mpl.rcParams["figure.figsize"] = (12, 3)
mpl.rcParams["axes.grid"] = False

# constnats
PRINT_RED = '\033[91m'

In [3]:
#--------------------------------------------------------------------------
# Database connection setup
#--------------------------------------------------------------------------

# get postgres environment variables
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_USERNAME = os.getenv("PG_USERNAME")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# check for missing environment variables
if PG_HOST == None or PG_PORT == None or PG_DB_NAME == None or PG_USERNAME == None or PG_PASSWORD == None:
    print(f'{PRINT_RED}[ ERROR ] Environment variables PG_HOST, PG_PORT, PG_DB_NAME, PG_USERNAME, or PG_PASSWORD not found.')

# hard-code table name
PG_TABLE = "bars_minute_eastern"

# connect to db and open a cursor to perform database operations
conn_string = f"postgresql://{PG_USERNAME}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB_NAME}"
db = create_engine(conn_string)
conn = db.connect()

In [4]:
#------------------------------------------------------------------------------
# Pull data from db and wrangle into correct shape
# (minute bars for NVDA from 09:00-14:00 from 2022/06/01-2022/07/01)
#------------------------------------------------------------------------------

# create empty dataframe to hold bar data
days_data_df = pd.DataFrame(columns=["date", "minute_bars"])

# create empty dataframe to hold timestamp data
days_timestamps_df = pd.DataFrame(columns=["date", "timestamp"])

# NOTE: db contains minute bars for NVDA,INTC from 2022/06/01 to 2022/07/01 (inclusive)
start_date = datetime.date(2022, 6, 1)
end_date = datetime.date(2022, 7, 1)

# loop over each date and get minute_bars from db
current_date = start_date
day_delta = datetime.timedelta(days=1)
while current_date <= end_date:
    # pull minute_bars for current_date
    day_minute_bars_df = pd.read_sql_query(
        sql=sql.text(f"select * from { PG_TABLE } where SYMBOL='NVDA' and date(TIMESTAMP)='{ current_date }'"),
        con=conn,
    )

    # add minute_bars to days_df if there is bar data
    if not day_minute_bars_df.empty:
        # drop symbol column
        day_minute_bars_df.drop("symbol", axis=1, inplace=True)

        #----------------------------------------------------------------------
        # Convert timestamps to day/week/year sin and cosine signals
        #----------------------------------------------------------------------

        # extract timestamps
        timestamps = pd.to_datetime(day_minute_bars_df.pop("timestamp"))

        timestamp_s = timestamps.map(pd.Timestamp.timestamp)

        seconds_per_day = 24 * 60 * 60
        seconds_per_week = seconds_per_day * 7
        seconds_per_year = seconds_per_week * 52.1429

        day_minute_bars_df["day-sin"] = np.sin(timestamp_s * (2 * np.pi / seconds_per_day))
        day_minute_bars_df["day-cos"] = np.cos(timestamp_s * (2 * np.pi / seconds_per_day))

        day_minute_bars_df["week-sin"] = np.sin(timestamp_s * (2 * np.pi / seconds_per_week))
        day_minute_bars_df["week-cos"] = np.cos(timestamp_s * (2 * np.pi / seconds_per_week))

        day_minute_bars_df["year-sin"] = np.sin(timestamp_s * (2 * np.pi / seconds_per_year))
        day_minute_bars_df["year-cos"] = np.cos(timestamp_s * (2 * np.pi / seconds_per_year))

        # insert day minute bars and timestamps to their respecive dataframes
        days_data_df.loc[len(days_data_df), days_data_df.columns] = current_date, day_minute_bars_df
        days_timestamps_df.loc[len(days_timestamps_df), days_timestamps_df.columns] = current_date, timestamps

    # go to next day
    current_date += day_delta

In [5]:
# -----------------------------------------------------------------------------
# Split into training/validation (70%), validation (20%), and test (10%) sets
# -----------------------------------------------------------------------------

n = len(days_data_df)  # total number of days

days_train_df = days_data_df[0 : int(n * 0.7)]
days_test_timestamps_df = days_timestamps_df[0 : int(n * 0.7)]

days_val_df = days_data_df[int(n * 0.7) : int(n * 0.9)]
days_test_timestamps_df = days_timestamps_df[int(n * 0.7) : int(n * 0.9)]

days_test_df = days_data_df[int(n * 0.9) :]
days_test_timestamps_df = days_timestamps_df[int(n * 0.9) :]

In [32]:
#-----------------------------------------------------------------------------
# Create and train a convolutional model
#-----------------------------------------------------------------------------

SEQUENCE_LENGTH = 30
SEQUENCE_STRIDE = 1
BATCH_SIZE = 32
CONV_WIDTH = 3
MAX_EPOCHS = 20

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=CONV_WIDTH, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear'),
])

conv_model.compile(
    loss=tf.keras.losses.MeanSquaredError(), # Computes the mean of squares of errors between labels and predictions.
    optimizer=tf.keras.optimizers.Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    metrics=[tf.keras.metrics.MeanAbsoluteError()], # Computes the mean of absolute difference between labels and predictions.
)

# loop over training days to fit the model
for _, day_data in days_train_df.iterrows():
    # Extract the minute bars for this day
    minute_bars = day_data["minute_bars"]

    # Extract the target variable (which we want to predict) from the minute bars
    targets = minute_bars["close"].values[SEQUENCE_LENGTH:]

    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=minute_bars[:-1], # use all but the last minute as input
        targets=targets,
        sequence_length=SEQUENCE_LENGTH,
        sequence_stride=SEQUENCE_STRIDE,
    )
    
    # Fit the model to the training data
    conv_model.fit(
        x=ds,
        batch_size=BATCH_SIZE,
        epochs=MAX_EPOCHS,
        verbose=1,
    )

Epoch 1/20
1/4 [======>.......................] - ETA: 1s - loss: 348306816.0000 - mean_absolute_error: 15594.3525

2023-04-27 23:59:06.749916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 36ms/step - loss: 154321216.0000 - mean_absolute_error: 9560.0479
Epoch 2/20
4/4 [==============================] - 0s 20ms/step - loss: 32502396.0000 - mean_absolute_error: 3535.9290
Epoch 3/20
4/4 [==============================] - 0s 39ms/step - loss: 38128308.0000 - mean_absolute_error: 4202.7104
Epoch 4/20
4/4 [==============================] - 0s 18ms/step - loss: 40964016.0000 - mean_absolute_error: 4440.6968
Epoch 5/20
4/4 [==============================] - 0s 19ms/step - loss: 21907602.0000 - mean_absolute_error: 3031.9253
Epoch 6/20
4/4 [==============================] - 0s 20ms/step - loss: 12095048.0000 - mean_absolute_error: 2257.7385
Epoch 7/20
4/4 [==============================] - 0s 15ms/step - loss: 12809914.0000 - mean_absolute_error: 2411.3171
Epoch 8/20
4/4 [==============================] - 0s 14ms/step - loss: 10612528.0000 - mean_absolute_error: 2151.8547
Epoch 9/20
4/4 [==============================] - 0s 14ms/step - l

In [33]:
#------------------------------------------------------------------------------
#
#-----------------------

for index, day_data in days_test_df.iterrows():
    # Extract the minute bars for this day
    minute_bars = day_data["minute_bars"]
    
    # Extract the target variable (which we want to predict) from the minute bars
    targets = minute_bars["close"].values[SEQUENCE_LENGTH:]

    dataset = tf.keras.utils.timeseries_dataset_from_array(
        data=minute_bars[:-1], # use all but the last minute as input
        targets=targets,
        sequence_length=SEQUENCE_LENGTH,
        sequence_stride=SEQUENCE_STRIDE,
        batch_size=BATCH_SIZE,
    )

    print(dataset.element_spec)
    
    # Evaluate the model on the test data
    loss, mae = conv_model.evaluate(x=dataset, verbose=0)
    print(f"Test loss: {loss}, Test MAE: {mae}")
    
    predictions = conv_model.predict(x=dataset, verbose=0, steps=None)

    print(predictions.shape)

    # plot_predictions(labels=targets, predictions=predictions, timestamps=days_test_timestamps_df[index])

(TensorSpec(shape=(None, None, 11), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))


2023-04-27 23:59:33.935786: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Test loss: 170876.296875, Test MAE: 256.9598083496094
(391, 28, 1)


2023-04-27 23:59:34.214391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


(TensorSpec(shape=(None, None, 11), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))
Test loss: 465513.09375, Test MAE: 405.3310241699219
(391, 28, 1)
(TensorSpec(shape=(None, None, 11), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))
Test loss: 169160.75, Test MAE: 281.47979736328125
(391, 28, 1)


In [ ]:
# #-----------------------------------------------------------------------------
# # Create baseline model (single-step, label is input)
# #-----------------------------------------------------------------------------

# example_day_minute_bars = days_data_df.iloc[0]["minute_bars"]

# single_step_window = WindowGenerator(
#     input_width=1,
#     label_width=1,
#     shift=1,
#     data=example_day_minute_bars,
#     label_columns=["close"],
# )

# baseline = Baseline(label_index=single_step_window.column_indices["close"])

# baseline.compile(
#     loss=tf.keras.losses.MeanSquaredError(),
#     metrics=[tf.keras.metrics.MeanAbsoluteError()],
# )

# performance = {}
# performance["Baseline"] = baseline.evaluate(
#     single_step_window.make_dataset(batch_size=32)
# )

# example_windows = tf.stack(
#     [
#         np.array(example_day_minute_bars[: single_step_window.total_window_size]),
#         np.array(example_day_minute_bars[1 : 1 + single_step_window.total_window_size]),
#         np.array(example_day_minute_bars[2 : 2 + single_step_window.total_window_size]),
#     ]
# )

# example_windows_inputs, example_windows_labels = single_step_window.split_windows(
#     example_windows
# )

# single_step_window.plot(
#     inputs=example_windows_inputs,
#     labels=example_windows_labels,
#     plot_col="close",
#     model=baseline,
# )


# current_date = start_date
# while current_date <= end_date:
#     # skip empty days (likely weekend or holiday)
#     if days_data_df[days_data_df["date"] == current_date].empty:
#         current_date += day_delta
#         continue

#     # pull minute_bars for current_date
#     day_minute_bars_df = days_data_df[days_data_df["date"] == current_date][
#         "minute_bars"
#     ].iloc[0]

#     current_date += day_delta